In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/2.part'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

directory_in = './file/1.url'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_request = crawler_series['Url']

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))

                        title = html.xpath('//head/title/text()')[0].strip()
                        break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            subtitle = html.xpath('//h2[@itemprop="brand"]/a/span[@itemprop="name"]/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            part_number = html.xpath('//dd[@itemprop="sku"]/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            dict_src, list_src = {}, html.xpath('//li[contains(@class, "productView-thumbnail")]/a[@class="productView-thumbnail-link"]/@href')
            for i, src in enumerate(list_src):
                dict_src[str(i)] = src.strip()

            # = = = = = = = = = = = = = = =

            type_, application_summary, list_dict, dict_count = '', '', json.loads(html.xpath('//div[@id="addon"]/@data-custom-fields')[0]), {}
            for dict_ in list_dict:
                key = dict_['name'].strip()
                if key == 'Part Type':
                    type_ = dict_['value'].strip()
                elif key == 'Application Summary':
                    application_summary = dict_['value'].strip()
                else:
                    if key in dict_count:
                        dict_count[key] += 1
                    else:
                        dict_count[key] = 1

            i, dict_info, dict_no = 0, {}, {}
            for dict_ in list_dict:
                key = dict_['name'].strip()
                if key not in ['Part Type', 'Application Summary']:
                    if dict_count[key] == 1:
                        dict_info[str(i)] = {key: dict_['value'].strip()}
                    else:
                        if key in dict_no:
                            dict_no[key] += 1
                        else:
                            dict_no[key] = 1
                        dict_info[str(i)] = {f'''{key}-{dict_no[key]}''': dict_['value'].strip()}
                    i += 1

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.append({'No': crawler_series['No'],
                                          'Type': type_,
                                          'Part Number': part_number,
                                          'Title': title,
                                          'Application Summary': application_summary,
                                          'Picture': '',
                                          'Url': crawler_series['Url'],
                                          'Subtitle': subtitle,
                                          'Json_Src': json.dumps(dict_src),
                                          'Json_Info': json.dumps(dict_info)})

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Request_Url'] = url_request

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                                 ascending=[True],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[True] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/25 16:59:25] - 1. https://www.cardone.com/product/windshield-wiper-motor-40-1000

[True] - [请求1次] - [剩余8条] - [crawler_1（50.00%）] - [2024/10/25 16:59:26] - 3. https://www.cardone.com/product/windshield-wiper-motor-40-10005

[True] - [请求1次] - [剩余7条] - [crawler_1（50.00%）] - [2024/10/25 16:59:27] - 6. https://www.cardone.com/product/windshield-wiper-motor-40-10009

[True] - [请求1次] - [剩余6条] - [crawler_1（50.00%）] - [2024/10/25 16:59:27] - 7. https://www.cardone.com/product/windshield-wiper-motor-40-1001

[True] - [请求1次] - [剩余5条] - [crawler_1（50.00%）] - [2024/10/25 16:59:28] - 10. https://www.cardone.com/product/windshield-wiper-motor-40-10015

[True] - [请求1次] - [剩余4条] - [crawler_1（50.00%）] - [2024/10/25 16:59:29] - 8. https://www.cardone.com/product/windshield-wiper-motor-40-10013

[True] - [请求1次] - [剩余3条] - [crawler_1（50.00%）] - [2024/10/25 16:59:29] - 4. https://www.cardone.com/product/windshield-wiper-motor-40-10006

[True]

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 2688.66it/s]

Done ~
